# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv

# Import API key
from api_keys import geoapify_key

In [21]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,50.83,71,12,1.99,NZ,1715134911
1,1,talnakh,69.4865,88.3972,12.20,90,41,5.55,RU,1715134911
2,2,wailua homesteads,22.0669,-159.3780,81.03,71,40,12.66,US,1715134911
3,3,edinburgh of the seven seas,-37.0676,-12.3116,61.29,92,100,44.52,SH,1715134912
4,4,whangarei,-35.7317,174.3239,60.10,60,24,6.42,NZ,1715134912


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [22]:
# Configure the map plot
city_humidity_points = gv.Points(city_data_df, ['Lng', 'Lat'], ['City', 'Country', 'Humidity'], label='City Information')

# Adjust point size based on humidity
map_plot = gv.tile_sources.CartoLight() * city_humidity_points.opts(
    size=gv.dim('Humidity') * 0.2,  # Adjust the factor as needed to scale point size
    color='blue',
    tools=['hover'],
    width=800,
    height=600,
    title="City Humidity Map"
)

# Display the map
map_plot

:Overlay
   .WMTS.I                  :WMTS   [Longitude,Latitude]
   .Points.City_Information :Points   [Lng,Lat]   (City,Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
# Filter the DataFrame based on ideal weather conditions
ideal_conditions_df = city_data_df[
    (city_data_df['Max Temp'] < 80) &
    (city_data_df['Max Temp'] > 12) &
    (city_data_df['Wind Speed'] < 20) &
    (city_data_df['Cloudiness'] < 100)
]

print(f"Number of cities with ideal conditions: {len(ideal_conditions_df)}")
ideal_conditions_df.head()

# Configure the map plot for the filtered DataFrame
city_humidity_points = gv.Points(ideal_conditions_df, ['Lng', 'Lat'], ['City', 'Country', 'Humidity'], label='City Information')

# Adjust point size based on humidity
map_plot = gv.tile_sources.CartoLight() * city_humidity_points.opts(
    size=gv.dim('Humidity') * 0.2,  # Adjust the factor as needed to scale point size
    color='blue',
    tools=['hover'],
    width=800,
    height=600,
    title="City Humidity Map"
)

# Display the map
map_plot

Number of cities with ideal conditions: 282


:Overlay
   .WMTS.I                  :WMTS   [Longitude,Latitude]
   .Points.City_Information :Points   [Lng,Lat]   (City,Country,Humidity)

### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Step 1: Use the Pandas copy function to create a DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Step 2: Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ''

# Step 3: Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,waitangi,NZ,-43.9535,-176.5597,71,
1,talnakh,RU,69.4865,88.3972,90,
2,wailua homesteads,US,22.0669,-159.3780,71,
3,edinburgh of the seven seas,SH,-37.0676,-12.3116,92,
4,whangarei,NZ,-35.7317,174.3239,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [34]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters
params = {
    'categories': 'accommodation.hotel',
    'limit': 1  # Limit to 1 result per query
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"point:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print(hotel_df)

Starting hotel search
waitangi - nearest hotel: No hotel found
talnakh - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
whangarei - nearest hotel: No hotel found
bako - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
ribeira grande - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
katsuura - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
stanley - nearest hotel: No hotel found
taltal - nearest hotel: No hotel found
saint-joseph - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
iqaluit - nearest hotel: No hotel found
longyearbyen - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
punta arenas - nearest hotel: No hotel found
puerto natales - nearest hot

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
# Configure the map plot for the filtered DataFrame with hotel information
hotel_points = gv.Points(hotel_df, ['Lng', 'Lat'], ['City', 'Country', 'Hotel Name'], label='City Information')

# Adjust point size based on humidity
map_plot_with_hotels = gv.tile_sources.CartoLight() * hotel_points.opts(
    size=gv.dim('Humidity') * 0.2,  # Adjust the factor as needed to scale point size
    color='blue',
    tools=['hover'],
    width=800,
    height=600,
    title="City Humidity Map with Hotels"
)

# Display the map with hotel information
map_plot_with_hotels

:Overlay
   .WMTS.I                  :WMTS   [Longitude,Latitude]
   .Points.City_Information :Points   [Lng,Lat]   (City,Country,Hotel Name)